<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/6.Gestion_de_datos/Agregaci%C3%B3n_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[Agregación de datos y operaciones de grupo](#scrollTo=BFynk27lvXm6)

>>[Actividades que veremos en este apartado](#scrollTo=EMGwZJuAv306)

>>>[Mecánica del GroupBy](#scrollTo=F1ju7vADwWy1)

>>>[Seleccionando una columna o subset de columnas](#scrollTo=_SAtsTx-1xjS)

>>>[Agrupando con dicts y series](#scrollTo=SFpRZZb23nOj)

>>>[Agrupación con funciones](#scrollTo=hNvhRjQg5IhT)

>>>[Data Aggregation](#scrollTo=tCCi7PrE5ecH)

>>>[Aplicación de columna inteligente y de funciones múltiples](#scrollTo=rbeK_M1blP_0)

>>[Aplicar: general dividir-aplicar-combinar](#scrollTo=K8sgtonjqmcI)

>>>[Análisis de cuantiles y buckets](#scrollTo=mFfqx7P0sbeq)

>>>[Rellenar valores perdidos con valores específicos de grupo](#scrollTo=tvKXddD9tV0C)

>>[Muestreo aleatorio y permutación](#scrollTo=vTsm2BuHv_SC)

>>>[Promedio ponderado grupal y correlación](#scrollTo=eX6B4Plhxr8P)

>>[Pivot Tables y tabulación cruzada](#scrollTo=3e5jR8qh0UMW)

>>>[Tabulaciones cruzadas (crosstab)](#scrollTo=gprrjJ0m12nf)



# Agregación de datos y operaciones de grupo

La categorización de un conjunto de datos y la aplicación de una función a cada grupo, ya sea una agregación o transformación, es un componente crítico del trabajo de análisis de datos. Después de cargar, fusionar y preparar un conjunto de datos, es posible que debas calcular estadísticas de grupo o posiblemente tablas dinámicas para fines de informes o visualización. Pandas proporciona una interfaz de grupo flexible, que te permite cortar, y resumir conjuntos de datos de forma natural.

Como verás, con la expresividad de Python y pandas, podemos realizar operaciones grupales bastante complejas utilizando cualquier función que acepte un objeto Pandas o una matriz NumPy.

## Actividades que veremos en este apartado

* Dividir un Dataframe en pedazos usando una o más claves (en forma de funciones, matrices o nombres de columna de DataFrame).

* Calcular estadísticas de resumen de grupo, como conteo, media o desviación estándar, o una función definida por el usuario.

* Aplicar transformaciones como normalización, regresión lineal, clasificación o selección de subconjuntos.

* Calcular tablas dinámicas y tabulaciones cruzadas.

* Realizar análisis de cuantiles y otros análisis de grupos estadísticos.

### Mecánica del GroupBy

Existe un término conocido entre los analistas que describe operaciones de grupo, *split-apply-combine*.

En la primera parte de este proceso dividimos dataframes o series (split) en grupos basados en una o más keys. Una vez realizado la división, realizamos la función *apply* a cada grupo, produciendo un nuevo valor.

Finalmente, tomamos el resultado de esas operaciones y las combinamos en un objeto.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*



In [62]:
#@title
import pandas as pd
import numpy as np
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,-0.671932,0.524839
1,a,two,-1.649940,-0.598616
2,b,one,0.382257,1.246345
3,b,two,1.206347,-0.092466
4,a,one,0.278931,0.587295


Supongamos que deseas calcular la *media* de la columna data1 usando las etiquetas de key1

In [63]:
#@title
grouped = df[['data1','data2']].groupby(df['key1']).mean()
print(grouped)

         data1     data2
key1                    
a    -0.680980  0.171173
b     0.794302  0.576940


In [64]:
#@title
grouped.mean()

data1    0.056661
data2    0.374056
dtype: float64

Aquí agrupamos los datos usando dos claves, y la Serie resultante ahora tiene un índice jerárquico.

In [65]:
#@title
media = df['data1'].groupby([df['key1'], df['key2']]).mean()
media

key1  key2
a     one    -0.196500
      two    -1.649940
b     one     0.382257
      two     1.206347
Name: data1, dtype: float64

In [66]:
#@title
media.unstack()

key2,one,two
key1,,
a,-0.196500,-1.649940
b,0.382257,1.206347


En el siguiente ejemplo todo el grupo de keys son series

In [67]:
#@title
prov = np.array(['Buenos Aires', 'Buenos Aires', 'Córdoba', 'Córdoba', 'Tucumán'])
anios = np.array([2005, 2005, 2005, 2006, 2006])

In [68]:
df

,key1,key2,data1,data2
0,a,one,-0.671932,0.524839
1,a,two,-1.649940,-0.598616
2,b,one,0.382257,1.246345
3,b,two,1.206347,-0.092466
4,a,one,0.278931,0.587295


In [69]:
#@title
print(df['data1'])
print(prov)
print(anios)

0   -0.671932
1   -1.649940
2    0.382257
3    1.206347
4    0.278931
Name: data1, dtype: float64
['Buenos Aires' 'Buenos Aires' 'Córdoba' 'Córdoba' 'Tucumán']
[2005 2005 2005 2006 2006]


In [70]:
#@title
df['data1'].groupby([prov, anios]).mean()

Buenos Aires  2005   -1.160936
Córdoba       2005    0.382257
              2006    1.206347
Tucumán       2006    0.278931
Name: data1, dtype: float64

### Seleccionando una columna o subset de columnas

La indexación de un objeto **GroupBy** creado a partir de un DataFrame con un nombre de columna o matriz de nombres de columna, genera un subconjunto de columnas para la agregación.

In [71]:
df.key1

0    a
1    a
2    b
3    b
4    a
Name: key1, dtype: object

In [72]:
#@title

df.groupby('key1')['data1'].mean() # Equivalente a df['data1'].groupby(df['key1'])
df.groupby('key1')['data2'].mean() # Equivalente a df[['data2']].groupby(df['key1'])

key1
a    0.171173
b    0.576940
Name: data2, dtype: float64

Especialmente para grandes conjuntos de datos, puede ser conveniente agregar solo unas pocas columnas. Por ejemplo, en el conjunto de datos anterior, para calcular promedios solo para la columna data2 y obtener el resultado como un DataFrame, podríamos escribir:

In [73]:
#@title
df.groupby(['key1', 'key2'])[['data1']].mean()

data1
key1 key2          
a    one  -0.196500
     two  -1.649940
b    one   0.382257
     two   1.206347

El objeto devuelto por esta operación de indexación es un DataFrame agrupado.

Será una lista o matriz o una Serie agrupada si solo se pasa un solo nombre de columna como escalar

In [74]:
#@title
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped.mean()

key1  key2
a     one     0.556067
      two    -0.598616
b     one     1.246345
      two    -0.092466
Name: data2, dtype: float64

In [75]:
var=s_grouped.mean()

In [76]:
var.unstack()

key2,one,two
key1,,
a,0.556067,-0.598616
b,1.246345,-0.092466


### Agrupando con dicts y series

Puede que necesites agrupar información existente en algo diferente a un arreglo. Consideremos el siguiente Dataframe:


In [77]:
#@title
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,0.138010,0.945005,0.241340,-0.740406,-1.797153
Steve,-1.977254,0.022752,0.591206,-0.608018,-0.176642
Wes,1.237390,-0.411805,-0.730249,-1.075798,0.055037
Jim,-0.251374,-0.842659,0.437469,-1.347912,1.462088
Travis,0.327544,-0.544882,-0.091066,0.768751,-0.400056


In [78]:
#@title
people.iloc[2:3, [1, 2]] = np.nan # Agrega un par de NaN
people

,a,b,c,d,e
Joe,0.138010,0.945005,0.241340,-0.740406,-1.797153
Steve,-1.977254,0.022752,0.591206,-0.608018,-0.176642
Wes,1.237390,NaN,NaN,-1.075798,0.055037
Jim,-0.251374,-0.842659,0.437469,-1.347912,1.462088
Travis,0.327544,-0.544882,-0.091066,0.768751,-0.400056


Supongamos que tenemos una lista de columnas que corresponden a ese Dataframe y queremos realizar una operacion **sum** entre las columnas por grupo

In [79]:
#@title
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

Ahora podemos construir un arreglo a partir del dict y se lo pasamos a la operación **groupby**, pero en cambio le pasamos directamente el dict como key.


In [80]:
#@title
by_column=people.groupby(mapping, axis=1).mean()
display(by_column)

,blue,red
Joe,-0.249533,-0.238046
Steve,-0.008406,-0.710382
Wes,-1.075798,0.646213
Jim,-0.455221,0.122685
Travis,0.338842,-0.205798


### Agrupación con funciones
El uso de las funciones de Python es una forma más genérica de definir un mapeo de grupo en comparación con un dict o Series.

**Cualquier función que se pase como clave de grupo se llamará una vez por valor de índice**, y los valores de retorno se utilizarán como nombres de grupo. Más concretamente, consideremos el DataFrame de ejemplo de la sección anterior, que tiene los nombres de las personas como valores de índice. Supongamos que deseas agrupar por la longitud de los nombres; Si bien podrías calcular una matriz de longitudes de cadena, es más simple simplemente pasar la función len:

In [81]:
#@title
people #Recordemos el Dataframe original

,a,b,c,d,e
Joe,0.138010,0.945005,0.241340,-0.740406,-1.797153
Steve,-1.977254,0.022752,0.591206,-0.608018,-0.176642
Wes,1.237390,NaN,NaN,-1.075798,0.055037
Jim,-0.251374,-0.842659,0.437469,-1.347912,1.462088
Travis,0.327544,-0.544882,-0.091066,0.768751,-0.400056


In [82]:
#@title
people.groupby(len).sum()

,a,b,c,d,e
3,1.124025,0.102346,0.678809,-3.164115,-0.280028
5,-1.977254,0.022752,0.591206,-0.608018,-0.176642
6,0.327544,-0.544882,-0.091066,0.768751,-0.400056




---


### Data Aggregation
Las agregaciones se refieren a cualquier transformación de datos que produce valores escalares a partir de matrices. Los ejemplos anteriores han utilizado varios de ellos, como el cálculo de promedio, la suma, etc.


Function name |	Description
------------- | -----------
count	| Número de valores no-NA en el grupo
sum	| Suma de valores no-NA
mean	| Media de valores no-NA
median	| Mediana aritmética de valores no-NA
std, var	| Desviación y varianza estándar imparcial (denominador n - 1)
min, max	| Mínimo y máximo de valores no-NA
prod	| Producto de valores no-NA
first, last	| Primer y último valores no-NA




Puedes usar agregaciones de tu propio diseño y, además, llamar a cualquier método que también esté definido en el objeto agrupado.

**Ejemplo**: Veamos por ejemplo un ejemplo de cálculo de cuantil sobre un dataframe agrupado.

Si bien el cuantil no se implementa explícitamente para GroupBy, es un método de la Serie y, por lo tanto, está disponible para su uso.  Internamente, GroupBy corta eficientemente la serie, llama a **quantile()** para cada pieza y luego ensambla esos resultados en el objeto de resultado:

In [83]:
#@title
df

,key1,key2,data1,data2
0,a,one,-0.671932,0.524839
1,a,two,-1.649940,-0.598616
2,b,one,0.382257,1.246345
3,b,two,1.206347,-0.092466
4,a,one,0.278931,0.587295


In [84]:
#@title
grouped = df.groupby('key1')
grouped

In [85]:
#@title
grouped['data1'].quantile(0.5)

#Análisis del resultado:
#1. Vemos que el cuantil del 50% para la clave "a", que tenia 3 valores, coincide con el valor del promedio de los extremos.
#2. En el caso de la clave "b" que tenia dos valores, coincide con el promedio de ambos.


key1
a   -0.671932
b    0.794302
Name: data1, dtype: float64

Puedes notar que algunos métodos como **describe** también funcionan, aunque no son agregaciones, estrictamente hablando

In [86]:
#@title
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.680980  0.964467 -1.649940 -1.160936 -0.671932 -0.196500   
b      2.0  0.794302  0.582720  0.382257  0.588279  0.794302  1.000324   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     0.278931   3.0  0.171173  0.667387 -0.598616 -0.036888  0.524839   
b     1.206347   2.0  0.576940  0.946682 -0.092466  0.242237  0.576940   

                          
           75%       max  
key1                      
a     0.556067  0.587295  
b     0.911642  1.246345

### Agregación de columna inteligente y de funciones múltiples

Volvamos al conjunto de datos de propinas de ejemplos anteriores. Después de cargarlo con read_csv, agregamos una columna de porcentaje de propina tip_pct

In [87]:
#@title
prop = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/tips.csv')
prop['tip_pct'] = prop['tip'] / prop ['total_bill']

In [88]:
#@title
prop[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Como hemos visto, agregar una Serie o todas las columnas de un dataframe de datos es una cuestión de utilizar el agregado con la función deseada o llamar a un método como **mean** o **std**.
Sin embargo, es posible que desees agregar usando una función diferente dependiendo de la columna, o múltiples funciones a la vez.

In [89]:
#@title
grouped = prop.groupby(['day', 'smoker', 'time'])
grouped

In [90]:
#@title
grouped_pct = grouped['tip_pct']
grouped_pct

Ten en cuenta que para estadísticas descriptivas como las de la Tabla que hemos compartido al comienzo, igual que cuando hicimos el agrupamiento por función **len** se puede pasar el nombre de la función como una cadena, en este caso **mean**

In [91]:
#@title
grouped_pct.mean()

day   smoker  time  
Fri   No      Dinner    0.139622
              Lunch     0.187735
      Yes     Dinner    0.165347
              Lunch     0.188937
Sat   No      Dinner    0.158048
      Yes     Dinner    0.147906
Sun   No      Dinner    0.160113
      Yes     Dinner    0.187250
Thur  No      Dinner    0.159744
              Lunch     0.160311
      Yes     Lunch     0.163863
Name: tip_pct, dtype: float64

Una manera equivalente de escribir lo mismo es realizar una **agregación**, que agregará tantas columnas como se le indique a la funcion **.agg()**:

In [92]:
#@title
grouped_pct.agg('mean')

day   smoker  time  
Fri   No      Dinner    0.139622
              Lunch     0.187735
      Yes     Dinner    0.165347
              Lunch     0.188937
Sat   No      Dinner    0.158048
      Yes     Dinner    0.147906
Sun   No      Dinner    0.160113
      Yes     Dinner    0.187250
Thur  No      Dinner    0.159744
              Lunch     0.160311
      Yes     Lunch     0.163863
Name: tip_pct, dtype: float64

In [93]:
#@title
grouped_pct.agg(['min','max'])

min       max
day  smoker time                      
Fri  No     Dinner  0.120385  0.155625
            Lunch   0.187735  0.187735
     Yes    Dinner  0.103555  0.263480
            Lunch   0.117735  0.259314
Sat  No     Dinner  0.056797  0.291990
     Yes    Dinner  0.035638  0.325733
Sun  No     Dinner  0.059447  0.252672
     Yes    Dinner  0.065660  0.710345
Thur No     Dinner  0.159744  0.159744
            Lunch   0.072961  0.266312
     Yes    Lunch   0.090014  0.241255

In [94]:
#@title
def peak_to_peak(arr):      # Función de agregación propia
  return arr.max() - arr.min()

Si pasas una lista de funciones o nombres de funciones, obtiene un DataFrame con nombres de columnas tomados de las funciones.

In [95]:
#@title
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker time                                    
Fri  No     Dinner  0.139622  0.017841      0.035239
            Lunch   0.187735       NaN      0.000000
     Yes    Dinner  0.165347  0.052676      0.159925
            Lunch   0.188937  0.050262      0.141580
Sat  No     Dinner  0.158048  0.039767      0.235193
     Yes    Dinner  0.147906  0.061375      0.290095
Sun  No     Dinner  0.160113  0.042347      0.193226
     Yes    Dinner  0.187250  0.154134      0.644685
Thur No     Dinner  0.159744       NaN      0.000000
            Lunch   0.160311  0.039222      0.193350
     Yes    Lunch   0.163863  0.039389      0.151240

¿Y si quisieramos ver también sobre cuántos datos se hace cada operación para cada key, qué agregaríamos?

In [96]:
#@title
grouped_pct.agg(['mean', 'std', peak_to_peak, 'count']) # ¿Qué falta?

mean       std  peak_to_peak  count
day  smoker time                                           
Fri  No     Dinner  0.139622  0.017841      0.035239      3
            Lunch   0.187735       NaN      0.000000      1
     Yes    Dinner  0.165347  0.052676      0.159925      9
            Lunch   0.188937  0.050262      0.141580      6
Sat  No     Dinner  0.158048  0.039767      0.235193     45
     Yes    Dinner  0.147906  0.061375      0.290095     42
Sun  No     Dinner  0.160113  0.042347      0.193226     57
     Yes    Dinner  0.187250  0.154134      0.644685     19
Thur No     Dinner  0.159744       NaN      0.000000      1
            Lunch   0.160311  0.039222      0.193350     44
     Yes    Lunch   0.163863  0.039389      0.151240     17



---
**Cambiar los nombres de columna resultantes de la agregación**:

No se necesita aceptar los nombres que GroupBy le da a las columnas. Si pasas una lista de tuplas (nombre, función), el primer elemento de cada tupla se usará como los nombres de columna de DataFrame.

In [97]:
#@title
grouped_pct.agg([('Promedio de tip%', 'mean'), ('Desvio de tip%', np.std)])

Promedio de tip%  Desvio de tip%
day  smoker time                                    
Fri  No     Dinner          0.139622        0.017841
            Lunch           0.187735             NaN
     Yes    Dinner          0.165347        0.052676
            Lunch           0.188937        0.050262
Sat  No     Dinner          0.158048        0.039767
     Yes    Dinner          0.147906        0.061375
Sun  No     Dinner          0.160113        0.042347
     Yes    Dinner          0.187250        0.154134
Thur No     Dinner          0.159744             NaN
            Lunch           0.160311        0.039222
     Yes    Lunch           0.163863        0.039389

Con un DataFrame tienes más opciones, ya que puedes especificar una lista de funciones para aplicar a todas las columnas o diferentes funciones por columna.

Para comenzar, supongamos que deseamos calcular las mismas tres estadísticas para las columnas tip_pct y total_bill

In [98]:
prop

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [99]:
#@title
columnas = ['tip_pct', 'total_bill'] # Ahora tenemos una lista de columnas a diferencia del ejemplo anterior donde solo seleccionabamos

# una columna y a esa columna le aplicabamos varias funciones
functions = ['count', 'mean', 'max'] # A cada una de las columnas de la lista le aplicaremos entonces varias funciones

result = grouped[columnas].agg(functions) # A las dos columnas del DF le aplicamos las tres funciones

result

tip_pct                     total_bill                  
                     count      mean       max      count       mean    max
day  smoker time                                                           
Fri  No     Dinner       3  0.139622  0.155625          3  19.233333  22.75
            Lunch        1  0.187735  0.187735          1  15.980000  15.98
     Yes    Dinner       9  0.165347  0.263480          9  19.806667  40.17
            Lunch        6  0.188937  0.259314          6  12.323333  16.27
Sat  No     Dinner      45  0.158048  0.291990         45  19.661778  48.33
     Yes    Dinner      42  0.147906  0.325733         42  21.276667  50.81
Sun  No     Dinner      57  0.160113  0.252672         57  20.506667  48.17
     Yes    Dinner      19  0.187250  0.710345         19  24.120000  45.35
Thur No     Dinner       1  0.159744  0.159744          1  18.780000  18.78
            Lunch       44  0.160311  0.266312         44  17.075227  41.19
     Yes    Lunch       17  0.163863  0.241255         17  19.190588  43.11

Ahora, supongamos que deseamos aplicar funciones potencialmente diferentes a una o más de las columnas. Para hacer esto, pasamos un dict a *agg* que contenga una asignación de nombres de columna a cualquiera de las especificaciones de funciones enumeradas hasta ahora

In [100]:
#@title
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker time               
Fri  No     Dinner   3.50     6
            Lunch    3.00     3
     Yes    Dinner   4.73    20
            Lunch    3.48    11
Sat  No     Dinner   9.00   115
     Yes    Dinner  10.00   104
Sun  No     Dinner   6.00   167
     Yes    Dinner   6.50    49
Thur No     Dinner   3.00     2
            Lunch    6.70   110
     Yes    Lunch    5.00    40

In [101]:
prop

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
...,...,...,...,...,...,...,...
239,29.03,5.92,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,No,Sat,Dinner,2,0.098204


In [102]:
#@title
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'total_bill' : ['min', 'max']})

tip_pct                               total_bill       
                         min       max      mean       std        min    max
day  smoker time                                                            
Fri  No     Dinner  0.120385  0.155625  0.139622  0.017841      12.46  22.75
            Lunch   0.187735  0.187735  0.187735       NaN      15.98  15.98
     Yes    Dinner  0.103555  0.263480  0.165347  0.052676       5.75  40.17
            Lunch   0.117735  0.259314  0.188937  0.050262       8.58  16.27
Sat  No     Dinner  0.056797  0.291990  0.158048  0.039767       7.25  48.33
     Yes    Dinner  0.035638  0.325733  0.147906  0.061375       3.07  50.81
Sun  No     Dinner  0.059447  0.252672  0.160113  0.042347       8.77  48.17
     Yes    Dinner  0.065660  0.710345  0.187250  0.154134       7.25  45.35
Thur No     Dinner  0.159744  0.159744  0.159744       NaN      18.78  18.78
            Lunch   0.072961  0.266312  0.160311  0.039222       7.51  41.19
     Yes    Lunch   0.090014  0.241255  0.163863  0.039389      10.34  43.11


---

## Apply

El método mas general de uso de GroupBy es **apply**.

Como se ilustra en la Figura, **apply** divide el objeto que se está manipulando en piezas, invoca la función pasada en cada pieza y luego intenta concatenar las piezas juntas.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/split-apply-combine.png)

*Fuente: Python for Data Analysis, 2nd Edition*

Supongamos que deseamos seleccionar los cinco valores principales de **tip_pct** por grupo. Primero, escribimos una función que seleccione las filas con los valores más grandes en una columna particular:

In [103]:
#@title
# Volvamos a trabajar con el dataframe original
prop.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [104]:
#@title
def top(df, n=5, column='tip_pct'):
  return df.sort_values(by=column)[-n:] # Está haciendo un sort por columna "tip_pct" y retornando las últimas "n" filas

In [105]:
#@title
top(prop, n=6) # Llamada a la función top y reemplaza n=5 de la funcion por n=6

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


Ahora, si agrupamos por fumador, por ejemplo, y llamamos a esta función, obtenemos lo siguiente:

In [106]:
#@title
prop.groupby('smoker').apply(top) # apply llama a la función top

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

¿Qué ha pasado aquí?

La función superior se llama en cada grupo de filas desde cada split del dataframe (el primer grupo es la agrupación smoker "Yes" y el segundo es la agrupación smoker "No") y luego los resultados se pegan usando *pandas.concat*, etiquetando las piezas con los nombres de los grupos.

Por lo tanto, el resultado tiene un índice jerárquico cuyo nivel interno contiene valores de índice del DataFrame original.

Si pasas una función a *apply* que toma otros argumentos o palabras clave, puedes pasarlos después de la función:

In [107]:
#@title
prop.groupby(['smoker', 'day']).apply(top, n=2, column='total_bill') # En este caso aplicamos sobre un agrupamiento de dos claves.

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  91        22.49   3.50     No   Fri  Dinner     2  0.155625
            94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  59        48.27   6.73     No   Sat  Dinner     4  0.139424
            212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  112       38.07   4.00     No   Sun  Dinner     3  0.105070
            156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 85        34.83   5.17     No  Thur   Lunch     4  0.148435
            142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  90        28.97   3.00    Yes   Fri  Dinner     2  0.103555
            95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  102       44.30   2.50    Yes   Sat  Dinner     3  0.056433
            170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  184       40.55   3.00    Yes   Sun  Dinner     2  0.073983
            182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 83        32.68   5.00    Yes  Thur   Lunch     2  0.152999
            197       43.11   5.00    Yes  Thur   Lunch     4  0.115982



---


### Análisis de cuantiles y buckets

Pandas tiene algunas herramientas, en particular *cut* y *qcut*, para dividir los datos en cubos con contenedores de tu elección o por cuantiles de muestra. La combinación de estas funciones con *groupby* hace que sea conveniente realizar análisis de buckets o cuantil en un conjunto de datos. Considere un conjunto de datos aleatorio simple y una categorización de bucket de igual longitud usando cut:

In [108]:
#@title
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
frame

,data1,data2
0,-1.118014,0.029232
1,-0.754477,0.421383
2,-1.438127,-0.358080
3,1.665255,-0.160153
4,1.830643,-0.834257
...,...,...
995,0.512110,0.544256
996,1.272218,0.692247
997,-0.315010,-1.396805
998,-1.134196,0.393143


In [109]:
#@title
quartiles = pd.cut(frame.data1, 4) # Cortamos los datos en 4 conjuntos

In [110]:
#@title
quartiles[:10]

0    (-1.513, 0.00882]
1    (-1.513, 0.00882]
2    (-1.513, 0.00882]
3       (1.531, 3.053]
4       (1.531, 3.053]
5     (0.00882, 1.531]
6    (-1.513, 0.00882]
7     (0.00882, 1.531]
8     (0.00882, 1.531]
9     (0.00882, 1.531]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-3.041, -1.513] < (-1.513, 0.00882] < (0.00882, 1.531] <
                                           (1.531, 3.053]]

El objeto  devuelto por *cut* se puede pasar directamente a *groupby*. Entonces podríamos calcular un conjunto de estadísticas para la columna data2 de la siguiente manera:

In [111]:
#@title
def get_stats(group):
  return {'min': group.min(), 'max': group.max(),
          'count': group.count(), 'mean': group.mean()}

In [112]:
#@title
grouped = frame['data2'].groupby(quartiles)

In [113]:
#@title
grouped.apply(get_stats) # que puedo agregar para que se vea mejor? ..un___..?

data1                   
(-3.041, -1.513]   min       -1.922286
                   max        2.406744
                   count     74.000000
                   mean       0.033328
(-1.513, 0.00882]  min       -2.746617
                   max        2.878714
                   count    415.000000
                   mean       0.022025
(0.00882, 1.531]   min       -2.929569
                   max        2.823365
                   count    449.000000
                   mean       0.007557
(1.531, 3.053]     min       -1.915329
                   max        1.746977
                   count     62.000000
                   mean      -0.110553
Name: data2, dtype: float64

In [114]:
#@title
# Haz tu magia
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.041, -1.513]",-1.922286,2.406744,74.0,0.033328
"(-1.513, 0.00882]",-2.746617,2.878714,415.0,0.022025
"(0.00882, 1.531]",-2.929569,2.823365,449.0,0.007557
"(1.531, 3.053]",-1.915329,1.746977,62.0,-0.110553


### Rellenar valores perdidos con valores específicos de grupo

En clases anteriores vimos que a veces simplemente haremos **dropna** pero otras veces necesitaremos reemplazar los datos faltantes (nulos) por valores convenientes.

*fillna* es la herramienta adecuada para usar; por ejemplo, aquí rellenamos los valores de NA con la media, como vimos previamente:

In [115]:
#@title
s = pd.Series(np.random.randn(6))

In [116]:
#@title
s[:3] = np.nan
s

0         NaN
1         NaN
2         NaN
3   -0.168524
4   -1.143001
5    0.137736
dtype: float64

In [117]:
#@title
s.fillna(s.mean())

0   -0.391263
1   -0.391263
2   -0.391263
3   -0.168524
4   -1.143001
5    0.137736
dtype: float64

**Supongamos que necesitas que el valor de relleno varíe según el grupo.**

Una forma de hacer esto es agrupar los datos y usar *apply* con una función que llame a *fillna* en cada fragmento de datos.

Aquí hay algunos datos de muestra sobre los estados de EE. UU. Divididos en regiones orientales y occidentales:

In [118]:
#@title
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [119]:
#@title
# group_key = ['East'] * 4 + ['West'] * 4 # Notación alternativa
group_key = ['East', 'East', 'East', 'East', 'West', 'West', 'West', 'West']

Ten en cuenta que la sintaxis ['Este'] * 4 produce una lista que contiene cuatro copias de los elementos en ['Este'].

In [120]:
#@title
data = pd.Series(np.random.randn(8), index=states)
data

Ohio          0.158297
New York     -1.776652
Vermont       0.660210
Florida       0.271284
Oregon        0.560355
Nevada        2.916785
California    0.089284
Idaho         1.533999
dtype: float64

In [121]:
#@title
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.158297
New York     -1.776652
Vermont            NaN
Florida       0.271284
Oregon        0.560355
Nevada             NaN
California    0.089284
Idaho              NaN
dtype: float64

In [122]:
#@title
data.groupby(group_key).mean() # Al hacer el promedio, NO contempla los nulos, no los suma.

East   -0.449024
West    0.324819
dtype: float64

In [123]:
#@title
fill_mean = lambda g: g.fillna(g.mean()) # Que hace esta funcion lambda?

In [124]:
#@title
data.groupby(group_key).apply(fill_mean)

<ipython-input-124-a809bb15d727>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio          0.158297
New York     -1.776652
Vermont      -0.449024
Florida       0.271284
Oregon        0.560355
Nevada        0.324819
California    0.089284
Idaho         0.324819
dtype: float64


---

## Muestreo aleatorio y permutación
Supongamos que deseas extraer una muestra aleatoria de un gran conjunto de datos para fines de simulación o alguna otra aplicación. Hay varias formas de realizar los "sorteos"; Aquí usamos el método de muestra para Series.

In [125]:
#@title
suits = ['H', 'S', 'C', 'D'] # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names) # Para cada letra itera por la cantidad de cartas

deck = pd.Series(card_val, index=cards)

Así que ahora tenemos una Serie de longitud 52 cuyo índice contiene nombres y valores de cartas que se usan en Blackjack y otros juegos (para simplificar las cosas, solo dejo que el as 'A' sea 1):


In [146]:
#@title
deck[:20]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
dtype: int64

In [127]:
#@title
def draw(deck, n=5): # Esta funcion recibe la serie "Deck" (el mazo) y devuelve una muestra random de "n" elementos
  return deck.sample(n)

In [128]:
#@title
draw(deck)

KC    10
KS    10
7S     7
5H     5
9H     9
dtype: int64

Supongamos que quieres dos cartas al azar de cada palo. Debido a que el palo es el último caracter de cada nombre de tarjeta, podemos agruparlo en base a esto y usar apply:

In [129]:
#@title
get_suit = lambda card: card[-1] # Tomo la última letra que es el palo

In [130]:
#@title
a = deck.groupby(get_suit)
a.apply(draw, n=2)

C  KC     10
   6C      6
D  10D    10
   8D      8
H  QH     10
   6H      6
S  6S      6
   4S      4
dtype: int64



---


### Un ejemplito de transformación y correlación entre columnas

Más adelante vamos a ver más a fondo el tema de correlación, cuando entremos a algoritmos de machine learning, pero veamos ahora una simple transformación.

Consideremos un conjunto de datos financieros originalmente obtenido de Yahoo! Finance que contiene precios al final del día para algunas acciones y el índice S&P 500 (el símbolo SPX):

In [131]:
#@title
close_px = pd.read_csv('https://raw.githubusercontent.com/al34n1x/DataScience/master/6.Gestion_de_datos/stocks.csv',
                       parse_dates=True, index_col=0)
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [132]:
#@title
close_px

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
...,...,...,...,...
2011-10-10,388.81,26.94,76.28,1194.89
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66


Una tarea de interés podría ser calcular un DataFrame que consta de las correlaciones anuales de los rendimientos diarios con SPX.

1° paso: Vamos a hacer una **transformación**:


In [133]:
#@title
rets = close_px.pct_change().dropna() #Calculamos el procentaje de cambio y eliminamos nulos
# Por defecto, la función pct_change, calcula el porcentaje de cambio entre el valor actual y el de la row inmediata anterior

Vemos que en lugar de los valores originales, ahora tenemos el porcentaje de cambio

¿Qué dato desapareció? ¿Tiene sentido?

In [134]:
#@title
rets

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
...,...,...,...,...
2011-10-10,0.051406,0.026286,0.036977,0.034125
2011-10-11,0.029526,0.002227,-0.000131,0.000544
2011-10-12,0.004747,-0.001481,0.011669,0.009795
2011-10-13,0.015515,0.008160,-0.010238,-0.002974


2° paso: Creamos una función que calcula la correlación por pares de cada columna con la columna 'SPX':

In [135]:
#@title
spx_corr = lambda x: x.corrwith(x['SPX']) #Esta funcion aplica a Dataframes exclusivamente y mide la correlación entre cada columna con la que se pasa como parámetro

In [136]:
#@title
get_year = lambda x: x.year
by_year = rets.groupby(get_year) # Agrupamos los porcentajes de cambio por año
resultado = by_year.apply(spx_corr)
'''
Llama a la funcion spx_corr para calcular la correlación de cada columna del dataframe contra
la columna 'SPX', luego de hacer la agrupacion por año.
'''
resultado

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


Si quisieramos ver la pinta que tiene una matriz de correlación completa para todas las columnas para alguno de los años, por ejemplo para 2003 (primer fila), podemos hacer:

In [137]:
#@title
resultado = resultado.loc[2003:2010,:]#Hago el loc que devuelve una serie y la transformo en dataframe
# Agrego "transpose" para obtener las filas como columnas
resultado

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0


In [138]:
#@title
resultado.corr() #Ahora si puedo correlacionar sobre el dataframe

,AAPL,MSFT,XOM,SPX
AAPL,1.000000,0.695178,0.889094,0.046341
MSFT,0.695178,1.000000,0.788995,0.704008
XOM,0.889094,0.788995,1.000000,0.161803
SPX,0.046341,0.704008,0.161803,1.000000




---


## Pivot Tables y tabulación cruzada

Una tabla dinámica es una herramienta de resumen de datos que se encuentra con frecuencia en programas de hojas de cálculo.

Agrega una tabla de datos por una o más claves, organizando los datos en un rectángulo con algunas de las claves de grupo a lo largo de las filas y algunas a lo largo de las columnas.

Las tablas dinámicas en Python con Pandas son posibles a través de la función *groupby*. DataFrame tiene un método *pivot_table* y también hay una función *pandas.pivot_table* de nivel superior. Además de proporcionar una interfaz conveniente para *groupby*, *pivot_table* puede agregar totales parciales, también conocidos como márgenes.

Volviendo al conjunto de datos de propinas, supongamos que deseamos calcular una tabla de promedios grupales:

In [139]:
#@title
prop.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [140]:
#@title
prop.pivot_table(index=['day', 'smoker'])
# En este caso estamos generando una agrupación con promedios por columna,
# y lo que obtenemos es un dataframe con índices jerárquicos

<ipython-input-140-342a98541322>:2: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  prop.pivot_table(index=['day', 'smoker'])


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

Ahora, supongamos que queremos agregar solo *tip_pct* y *size*, y además agrupar por tiempo.

Pondremos fumador en las columnas de la tabla y día en las filas:

In [141]:
#@title
prop.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [142]:
#@title
prop.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0) # Si hay NaN podemos usar fill_value

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706


### Tabulaciones cruzadas (crosstab)
Una tabulación cruzada es un caso especial de una tabla dinámica que **calcula las frecuencias de grupo**. Aquí hay un ejemplo:

In [143]:
#@title
pd.crosstab([prop.time, prop.day], prop.smoker)

smoker       No  Yes
time   day          
Dinner Fri    3    9
       Sat   45   42
       Sun   57   19
       Thur   1    0
Lunch  Fri    1    6
       Thur  44   17

Podríamos aumentar esta tabla para incluir totales parciales pasando 'margins=True'. Esto tiene el efecto de agregar todas las etiquetas de fila y columna, siendo los valores correspondientes las estadísticas de grupo para todos los datos dentro de un solo nivel

In [144]:
#@title
df_cross = pd.crosstab([prop.time, prop.day], prop.smoker, margins=True)
df_cross

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [145]:
#@title
#Si se quiere acceder a un elemento dentro de un indice jerarquico,
# refinamos por columna y luego por indice con la jerarquia
df_cross = df_cross[['All']].loc['Dinner','Fri']
df_cross

smoker
All    12
Name: (Dinner, Fri), dtype: int64